In [382]:
import numpy as np
import os
import cv2
from glob import glob

In [383]:
# Prepare results directory
results_dir = "./results"
for folder in glob("*", root_dir="samples/"):
    os.makedirs(f"{results_dir}/{folder}", exist_ok=True)

In [384]:
# Read images into original_imgs
img_files = glob("*/*", root_dir="samples/", recursive=True)
original_images = []
for file in img_files:
    img = cv2.imread(f"samples/{file}")
    img = cv2.resize(img, (512, 512))
    original_images.append(img)

In [385]:
def wb(channel, perc = 5):
    mi, ma = (np.percentile(channel, perc), np.percentile(channel,100.0-perc))
    channel = np.uint8(np.clip((channel-mi)*255.0/(ma-mi+0.01), 0, 255))
    return channel
# Do Stuff
result_imgs = []
for img in original_images:
    
    # img = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
    
    # patches = []
    # patch_size = 512
    # for i in range(0, img.shape[0], patch_size):
    #     for j in range(0, img.shape[1], patch_size):
    #         patches.append(img[i:i+patch_size, j:j+patch_size])
    
    # for patch in patches:
    #     patch[:,:,0] = cv2.createCLAHE(clipLimit=5.0, tileGridSize=(2,2)).apply(patch[:, :, 0])
    #     # patch[:, :, 0] = cv2.equalizeHist(patch[:, :, 0])
    #     # patch[:, :, 0] = wb(patch[:, :, 0])
    #     # patch[:, :, 1] = wb(patch[:, :, 1])
    #     # patch[:, :, 2] = wb(patch[:, :, 2])
    # img = cv2.cvtColor(img, cv2.COLOR_YCrCb2BGR)
    # img = cv2.medianBlur(img, 5)

    #threshold
    # img = cv2.GaussianBlur(img, (3,3), 10)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
    # hsv[:,:,0] = wb(hsv[:, :, 0])
    hsv[:,:,1] = wb(hsv[:, :, 1])
    hsv = cv2.cvtColor(hsv, cv2.COLOR_YCrCb2BGR)
    hsv[np.argmax(hsv, axis=2) == 0] = 0
    hsv[np.argmax(hsv, axis=2) == 1] = 0
    hsv[np.argmax(hsv, axis=2) == 2] = 255
    # hsv[:,:,2] = wb(hsv[:, :, 2])
    # hsv[:, :, 0] = cv2.equalizeHist(hsv[:, :, 0])
    # Define the color range for the hand in HSV
    lower_color = np.array([0, 130, 75])
    upper_color = np.array([200, 163, 150])

    # Create a binary mask using color thresholding
    color_mask = cv2.inRange(hsv, lower_color, upper_color)
    color_mask = hsv

    # # Convert the image to grayscale
    # gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # # Apply GaussianBlur to the grayscale image
    # blurred = cv2.GaussianBlur(gray, (5, 5), 7)

    # # Apply adaptive thresholding to the grayscale image
    # adaptive_threshold = cv2.adaptiveThreshold(
    #     blurred,
    #     255,  # Max pixel value after thresholding
    #     cv2.ADAPTIVE_THRESH_MEAN_C,
    #     cv2.THRESH_BINARY,
    #     13,   # Block size
    #     2    # Constant
    # )

    # Combine the color mask and adaptive threshold
    # combined_mask = cv2.bitwise_and(color_mask, adaptive_threshold)
    combined_mask = color_mask
    # combined_mask = cv2.close
    # combined_mask = cv2.Sobel(combined_mask, 3, ksize=7, dx=1, dy=1)
    combined_mask = cv2.Canny(combined_mask, 3, 9, apertureSize=3)
    # combined_mask = cv2.erode(combined_mask, np.ones((2, 2)))
    # hsv = cv2.cvtColor(hsv, cv2.COLOR_YCrCb2BGR)
    result_imgs.append(hsv)

In [386]:
# Classifier

In [387]:
# Save results
i=0
for img in result_imgs:
    cv2.imwrite(f"{results_dir}/{img_files[i]}", img)
    i+=1